In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2022 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

/var/folders/33/syn88g8j53n7x_0ysjcp508w0000gn/T/ipykernel_78157/1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [9]:
df.tail(30)

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
302,DECEMBER,1.0,A Hollywood Christmas,HBO Max / Warner Bros. Pictures,Alex Ranarivelo (director); John Ducey (screen...,NaN,[240]
303,DECEMBER,2.0,Violent Night,Universal Pictures / 87North Productions,"Tommy Wirkola (director); Pat Casey, Josh Mill...",NaN,[251]
304,DECEMBER,2.0,Emancipation,Apple TV+ / Apple Studios / Westbrook Studios ...,Antoine Fuqua (director); William N. Collage (...,NaN,[252]
305,DECEMBER,2.0,Women Talking,Orion Pictures / Plan B Entertainment,Sarah Polley (director/screenplay); Rooney Mar...,NaN,[253]
306,DECEMBER,2.0,Marlowe,Open Road Films,Neil Jordan (director); William Monahan (scree...,NaN,[254]
307,DECEMBER,2.0,Spoiler Alert,Focus Features,Michael Showalter (director); David Marshall G...,NaN,[255]
308,DECEMBER,2.0,Diary of a Wimpy Kid: Rodrick Rules,Disney+ / Walt Disney Pictures / Bardel Entert...,Luke Cormican (director); Jeff Kinney (screenp...,NaN,[256]
309,DECEMBER,2.0,Darby and the Dead,Hulu / 20th Century Studios,"Silas Howard (director); Wenonah Wilms, Becca ...",NaN,NaN
310,DECEMBER,2.0,Lady Chatterley's Lover,Netflix / Blueprint Pictures,Laure de Clermont-Tonnerre (director); David M...,NaN,[220]
311,DECEMBER,2.0,The Eternal Daughter,A24 / BBC Film / Element Pictures,Joanna Hogg (director/screenplay); Tilda Swinton,NaN,NaN


In [10]:
df_2022 = df[['Title','Cast and crew']]

In [11]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,American Siege,Edward Drake (director/screenplay); Timothy V....
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
...,...,...
327,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...
328,NaN,NaN
329,NaN,NaN
330,NaN,NaN


In [12]:
!pip install tmdbv3api

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'ddc46900c7ab22df7cb7c606418b5216'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [15]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

/var/folders/33/syn88g8j53n7x_0ysjcp508w0000gn/T/ipykernel_78157/2369738557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))


In [16]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Thriller Crime Drama
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
...,...,...,...
327,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama
328,NaN,NaN,NaN
329,NaN,NaN,NaN
330,NaN,NaN,NaN


In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

/var/folders/33/syn88g8j53n7x_0ysjcp508w0000gn/T/ipykernel_78157/3796821595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))


In [19]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [21]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

/var/folders/33/syn88g8j53n7x_0ysjcp508w0000gn/T/ipykernel_78157/2929607607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [22]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [23]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

/var/folders/33/syn88g8j53n7x_0ysjcp508w0000gn/T/ipykernel_78157/2384233428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [24]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [25]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [26]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Thriller Crime Drama,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
...,...,...,...,...,...,...,...
327,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick
328,NaN,NaN,NaN,nan,nan,NaN,NaN
329,NaN,NaN,NaN,nan,nan,NaN,NaN
330,NaN,NaN,NaN,nan,nan,NaN,NaN


In [28]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [29]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [30]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN,Action Thriller Crime Drama,American Siege
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
...,...,...,...,...,...,...
327,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Drama,"Alice, Darling"
328,nan,nan,NaN,NaN,NaN,NaN
329,nan,nan,NaN,NaN,NaN,NaN
330,nan,nan,NaN,NaN,NaN,NaN


In [32]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [33]:
new_df22.isna().sum()

director_name     0
actor_1_name      0
actor_2_name     11
actor_3_name     38
genres            5
movie_title       4
comb             39
dtype: int64

In [35]:
new_df22 = new_df22.dropna(how='any')

In [36]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [37]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

/var/folders/33/syn88g8j53n7x_0ysjcp508w0000gn/T/ipykernel_78157/2732328091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title'] = new_df22['movie_title'].str.lower()


In [38]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
5,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Comedy Family Fantasy Adventure,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
6,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Thriller,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
323,Damien Chazelle,Brad Pitt,Margot Robbie,Diego Calva,Drama Comedy,babylon,Brad Pitt Margot Robbie Diego Calva Damien Cha...
324,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime Thriller Mystery Horror,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
325,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Drama Comedy,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...
326,Juel Taylor,John Boyega,Teyonah Parris,Kiefer Sutherland,Science Fiction Comedy Mystery,they cloned tyrone,John Boyega Teyonah Parris Kiefer Sutherland J...


In [39]:
old_df = pd.read_csv('/Users/merhawitsegay/Downloads/self_study/github/ds_ml_02/final_data.csv')

In [40]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5875,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
5876,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5877,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5878,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [41]:
final_df = old_df.append(new_df22,ignore_index=True)

/var/folders/33/syn88g8j53n7x_0ysjcp508w0000gn/T/ipykernel_78157/1166909226.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df22,ignore_index=True)


In [42]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6168,Damien Chazelle,Brad Pitt,Margot Robbie,Diego Calva,Drama Comedy,babylon,Brad Pitt Margot Robbie Diego Calva Damien Cha...
6169,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime Thriller Mystery Horror,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
6170,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Drama Comedy,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...
6171,Juel Taylor,John Boyega,Teyonah Parris,Kiefer Sutherland,Science Fiction Comedy Mystery,they cloned tyrone,John Boyega Teyonah Parris Kiefer Sutherland J...


In [45]:
main_data2 = pd.read_csv('/Users/merhawitsegay/Downloads/self_study/github/ds_ml_02/main_data.csv')

In [46]:
main_data2 =pd.concat([final_df,main_data],ignore_index=True)

In [48]:
main_data2.tail()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
12293,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Family Comedy Fantasy Drama,soul,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
12294,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
12295,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
12296,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...
12297,Kornél Mundruczó,Vanessa Kirby,Shia LaBeouf,Molly Parker,Drama,pieces of a woman,Vanessa Kirby Shia LaBeouf Molly Parker Kornél...


In [49]:
main_data2.to_csv('main_data.csv',index=False)

In [ ]:
main_data